In [1]:
import os
import shutil
import glob
from tqdm import tqdm

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3,MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [3]:
Raw_DIR= r'Z:\major project\MRL eye dataset\mrlEyes_2018_01'
for dirpath, dirname, filenames in os.walk(Raw_DIR):
    for i in tqdm([f for f in filenames if f.endswith('.png')]):
        if i.split('_')[4]=='0':
            
            shutil.copy(src=dirpath+'/'+i, dst=r'Z:\major project\MRL eye dataset\Prepared data\Closed eyes')
        
        elif i.split('_')[4]=='1':
            shutil.copy(src=dirpath+'/'+i, dst=r'Z:\major project\MRL eye dataset\Prepared data\open eyes')


0it [00:00, ?it/s]
100%|██████████| 10257/10257 [01:05<00:00, 157.02it/s]


In [4]:
raw_data = 'Z:/major project/MRL eye dataset/mrlEyes_2018_01'
for dirpath, dirname, filename in os.walk(raw_data):
    for file in tqdm([f for f in filename if f.endswith('.png')]):
        if file.split('_')[4] == '0':
            path='Z:/major project/MRL eye dataset/data/train/closed'
            if not os.path.exists(path):
                os.makedirs(path)
            shutil.copy(src=dirpath + '/' + file, dst= path)
        elif file.split('_')[4] == '1':
            path='Z:/major project/MRL eye dataset/data/train/open'
            if not os.path.exists(path):
                os.makedirs(path)
            shutil.copy(src=dirpath + '/' + file, dst= path)        

0it [00:00, ?it/s]
100%|██████████| 10257/10257 [01:04<00:00, 159.23it/s]


In [5]:
batchsize=8

In [6]:
def create_test_closed(source, destination, percent):
    '''
    divides closed eyes images into given percent and moves from
    source to destination.
    
    Arguments:
    source(path): path of source directory
    destination(path): path of destination directory
    percent(float): percent of data to be divided(range: 0 to 1)
    '''
    path, dirs, files_closed = next(os.walk(source))
    file_count_closed = len(files_closed)
    percentage = file_count_closed * percent
    to_move = random.sample(glob.glob(source + "/*.png"), int(percentage))

    for f in enumerate(to_move):
        if not os.path.exists(destination):
            os.makedirs(destination)
        shutil.move(f[1], destination)
    print(f'moved {int(percentage)} images to the destination successfully.')    

In [7]:
def create_test_open(source, destination, percent):
    '''
    divides open eyes images into given percent and moves from
    source to destination.
    
    Arguments:
    source(path): path of source directory
    destination(path): path of destination directory
    percent(float): percent of data to be divided(range: 0 to 1)
    '''
    path, dirs, files_open = next(os.walk(source))
    file_count_open = len(files_open)
    percentage = file_count_open * percent
    to_move = random.sample(glob.glob(source + "/*.png"), int(percentage))

    for f in enumerate(to_move):
        if not os.path.exists(destination):
            os.makedirs(destination)
        shutil.move(f[1], destination)
    print(f'moved {int(percentage)} images to the destination successfully.')

In [8]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [9]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory('Z:/major project/MRL eye dataset/data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory('Z:/major project/MRL eye dataset/data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 67919 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory('Z:/major project/MRL eye dataset/data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 84898 images belonging to 2 classes.


In [17]:
bmodel = MobileNet(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)), alpha=1.0)

hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)


model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False


In [18]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 40, 40, 32)        864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 40, 40, 32)        128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 40, 40, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 40, 40, 32)        288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 40, 40, 32)        128 

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [20]:
checkpoint = ModelCheckpoint('Z:/major project/Driver-Drowsiness-Detection-using-Deep-Learning-main/models',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=2)

C:\Users\tangu\AppData\Local\Temp\ipykernel_7140\2628715689.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/2


8489/8489 [==============================] - ETA: 0s - loss: 0.2235 - accuracy: 0.9143
Epoch 1: val_loss improved from inf to 0.30441, saving model to Z:/major project/Driver-Drowsiness-Detection-using-Deep-Learning-main\models
INFO:tensorflow:Assets written to: Z:/major project/Driver-Drowsiness-Detection-using-Deep-Learning-main\models\assets


INFO:tensorflow:Assets written to: Z:/major project/Driver-Drowsiness-Detection-using-Deep-Learning-main\models\assets


8489/8489 [==============================] - 763s 90ms/step - loss: 0.2235 - accuracy: 0.9143 - val_loss: 0.3044 - val_accuracy: 0.8811 - lr: 0.0010
Epoch 2/2
8489/8489 [==============================] - ETA: 0s - loss: 0.1833 - accuracy: 0.9293
Epoch 2: val_loss did not improve from 0.30441
8489/8489 [==============================] - 782s 92ms/step - loss: 0.1833 - accuracy: 0.9293 - val_loss: 0.3424 - val_accuracy: 0.8856 - lr: 0.0010


In [22]:
loss_tr,acc_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

 241/8490 [..............................] - ETA: 6:27 - loss: 0.1844 - accuracy: 0.9315

KeyboardInterrupt: 

In [ ]:
loss_vr,acc_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
loss_test,acc_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

In [ ]:
pip install pygame

In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model('Z:\major project\Driver-Drowsiness-Detection-using-Deep-Learning-main\models')

In [ ]:
mixer.init()
sound= mixer.Sound(r"Z:\major project\Driver-Drowsiness-Detection-using-Deep-Learning-main\alarm.wav")
cap = cv2.VideoCapture(0)
Score = 0
while True:
    ret, frame = cap.read()
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
    eyes= eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
    
    cv2.rectangle(frame, (0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color= (255,0,0), thickness=3 )
        
    for (ex,ey,ew,eh) in eyes:
        #cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color= (255,0,0), thickness=3 )
        
        # preprocessing steps
        eye= frame[ey:ey+eh,ex:ex+ew]
        eye= cv2.resize(eye,(80,80))
        eye= eye/255
        eye= eye.reshape(80,80,3)
        eye= np.expand_dims(eye,axis=0)
        # preprocessing is done now model prediction
        prediction = model.predict(eye)
        
        # if eyes are closed
        if prediction[0][0]>0.30:
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            Score=Score+1
            if(Score>15):
                try:
                    sound.play()
                except:
                    pass
            
        # if eyes are open
        elif prediction[0][1]>0.90:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)      
            cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            Score = Score-1
            if (Score<0):
                Score=0
            
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
